# Домашнее задание 2 по ML
Нужно написать python класс градиентного бустинга и побить другую модель на предоставленном baseline

## Критерии оценки
- Ваш ноутбук будет запущен через ```run all``` - он не должен упасть (допускается падение из-за отсутствия библиотеки, которую можно поставить через pip install)
- Вот этот код (внизу ноутбука) ```assert imp_my_little_model > imp_baseline_model``` не вызвал ошибок (успешно отработал)

- реализованы следующие гиперпараметры
  - вы реализовали гиперпараметр ```learning_rate```
  - вы реализовали гиперпараметр ```n_estimators```
  - вы реализовали гиперпараметр ```max_depth```
  - вы реализовали гиперпараметр ```bagging_fraction```

- Вы реализовали [Huber loss function](https://ru.wikipedia.org/wiki/%D0%A4%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F_%D0%BF%D0%BE%D1%82%D0%B5%D1%80%D1%8C_%D0%A5%D1%8C%D1%8E%D0%B1%D0%B5%D1%80%D0%B0) - она записана как отдельная def функция вне класса - и используется в вашем классе для расчета

----
*Для успешной сдачи дз нужно выполнить полностью каждый пункт выше*

- оценка 5 будет поставлена, если каждый пункт выполнен без недочетов
- оценка 4 будет поставлена, если будет найден один недочет
- незачет будет пославлен, если недочетов будет два или более
- незачет будет пославлен, если какой либо пункт не выполнен


In [ ]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
!pip install shap

import numpy as np
import shap

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
np.random.seed(42)

In [ ]:
data, target = shap.datasets.california()
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

### Не меняйте название для предскзаний preds_my_little_model, иначе не получится сдать это ДЗ (сломается код)

Некоторые правила
- Нельзя использовать никакие другие алгоритмы моделей внутри вашего класса, кроме DecisionTreeRegressor.
- Код вашего бустинга должен быть написан в классе, у класса должно быть два ожидаемых метода : ```fit``` и ```predict```.
- Нельзя менять датасет (и модифицировать тоже, например заполнять nan или применять scaler) или baseline модель
- Нельзя поднимать число n_estimators вашей модели выше 100 (чтобы результат был сравним с моделью-конкурентом ```GradientBoostingRegressor```)

# *это место для вашего кода* ↓↓↓


In [ ]:
def huber_loss(y_true, y_pred, delta=1.0):
    diff = np.abs(y_true - y_pred) # Вычисляем абсолютную разницу между истинными значениями и предсказанными

    quadratic_part = np.where(diff <= delta, 0.5 * diff ** 2, delta * (diff - 0.5 * delta)) # Штрафуем ошибки линейно, если абсолютная разница меньше или равна delta

    return np.mean(quadratic_part) # Возвращаем среднее значение потерь

In [ ]:
class MyGradBoosting:
    def __init__(
      self,
      learning_rate=0.01,
      n_estimators=100,
      max_depth=15,
      random_state=0,
      bagging_fraction=0.75
  ):
      self.learning_rate = learning_rate
      self.n_estimators = n_estimators
      self.max_depth = max_depth
      self.random_state = random_state
      self.bagging_fraction = bagging_fraction
      self.trees = []

    def fit(self, X, y):
        self.trees = [] # Очищаем список деревьев
        residuals = y.copy() # Копируем целевую переменную, чтобы не изменять оригинал

        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            sample_indices = np.random.choice(len(X), size=int(self.bagging_fraction * len(X)), replace=False)
            X_sampled = X_train.iloc[sample_indices]  # Используем iloc для извлечения строк по индексам
            residuals_sampled = residuals[sample_indices]
            tree.fit(X_sampled, residuals_sampled)

            self.trees.append(tree)
            predictions = tree.predict(X)
            residuals -= self.learning_rate * huber_loss(y, predictions)

    def predict(self, X):
        predictions = np.zeros(len(X))

        for tree in self.trees:
            predictions += self.learning_rate * tree.predict(X)

        return predictions # Возвращаем вектор предсказанных значений

In [ ]:
my_little_model = MyGradBoosting()
my_little_model.fit(X_train, y_train)

preds_my_little_model = my_little_model.predict(X_test)

In [ ]:
# самопроверки
assert preds_my_little_model.shape == y_test.shape, 'что-то не так с выходным размером предикта'

# *это место для вашего кода* ↑↑↑

### *ниже ничего менять не нужно*
## Это класс судья - он решит, какая модель оказалась лучше, ваша, или GradientBoostingRegressor из sklearn
Если ячейка ниже завершилась ошибкой, нужно поменять код вашей модели и попробовать еще раз, до тех пор, пока не получите сообщение "Ура, получилось!"

In [ ]:
baseline_model = GradientBoostingRegressor(random_state=4, verbose=0)
baseline_model = baseline_model.fit(X_train, y_train)
preds_baseline_model = baseline_model.predict(X_test)
print('mape - ваша модель', mean_absolute_percentage_error(y_test, preds_my_little_model))
print('mape - baseline', mean_absolute_percentage_error(y_test, preds_baseline_model))

final_estimator = RandomForestRegressor(random_state=16)
final_estimator = final_estimator.fit(
    np.hstack((preds_baseline_model.reshape(-1, 1), preds_my_little_model.reshape(-1, 1))),
    y_test
)

imp_baseline_model, imp_my_little_model = final_estimator.feature_importances_
result_message = f"baseline важность: {imp_baseline_model:0.3f}; важность вашей модели: {imp_my_little_model:0.3f}"

assert imp_my_little_model > imp_baseline_model,  f'попробуй еще раз: {result_message}'
print('Ура, получилось!',  result_message)

mape - ваша модель 0.18370268368520962
mape - baseline 0.2152446498010688
Ура, получилось! baseline важность: 0.159; важность вашей модели: 0.841
